## Image Captioning V3

In [1]:
from picturedata.generator import DataGenerator
from keras.models import Model
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, merge, Activation, Flatten, GRU, Input
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


Here the final model is created and trained. The model is based on https://github.com/anuragmishracse/caption_generator

In [2]:
generator = DataGenerator()

Total samples : 383454
Vocabulary size: 8256
Maximum caption length: 40
Image count: 30000


The structure of final model. Two networks, one for images and another for texts, merged into one to produce output. The text network uses RNN to process text, in this case, a LSTM.

In [13]:
embedding_dim = 128
#image_input = Input(shape=(1000,))
image_model = Dense(embedding_dim, input_dim=1000, activation='relu')(image_input)

image_model = RepeatVector(generator.getMaxCapLen())(image_model)

language_input = Input(shape=(generator.getMaxCapLen(),))
language_model = Embedding(generator.getVocabSize(), 256, input_length=generator.getMaxCapLen())(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
#language_model.add(GRU(128, return_sequences=True))
language_model = TimeDistributed(Dense(embedding_dim))(language_model)

output = merge([image_model, language_model], mode='concat')
output = LSTM(1000, return_sequences=False)(output)
#model.add(GRU(256, return_sequences=False))
output = Dense(generator.getVocabSize())(output)
output = Activation('softmax')(output)

model = Model(inputs=[image_input, language_input], outputs=[output])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]


In [14]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 40)            0                                            
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 4096)          0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 40, 256)       2113536     input_7[0][0]                    
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 128)           524416      input_5[0][0]                    
___________________________________________________________________________________________

Using checkpoint to get the best trained weight

In [16]:
file_name = 'weights-improvement-{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit_generator(generator.generate(batch_size=32), steps_per_epoch=generator.total_samples/32, epochs=20, verbose=1, callbacks=callbacks_list)
try:
    model.save('Models/WholeModel.h5', overwrite=True)
    model.save_weights('Models/Weights.h5',overwrite=True)
except:
    print "Error in saving model."

Epoch 1/20
Generating data...


ValueError: Error when checking input: expected input_5 to have shape (None, 4096) but got array with shape (31, 1000)